In [1]:
import numpy as np
import pandas as pd
import requests
import time
import json
import psycopg2
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
from psycopg2.extensions import register_adapter, AsIs
psycopg2.extensions.register_adapter(np.int64, psycopg2._psycopg.AsIs)


In [3]:
WEBSITE = "https://www.jiandaoyun.com"
appId = '617643f8308c2f000941f21b'
entryId = '617fc35d7ffed400078e5b08'
api_key = 'nqLEWktb51m3oYbYOZa5a2Ai6Lp8BHT7'
url_get_widgets = WEBSITE + '/api/v1/app/' + appId + '/entry/' + entryId + '/widgets'
url_get_data = WEBSITE + '/api/v1/app/' + appId + '/entry/' + entryId + '/data'
url_retrieve_data = WEBSITE + '/api/v1/app/' + appId + '/entry/' + entryId + '/data_retrieve'
url_update_data = WEBSITE + '/api/v1/app/' + appId + '/entry/' + entryId + '/data_update'
url_create_data = WEBSITE + '/api/v1/app/' + appId + '/entry/' + entryId + '/data_create'
url_delete_data = WEBSITE + '/api/v1/app/' + appId + '/entry/' + entryId + '/data_delete'
api_key = api_key

In [6]:
def get_req_header( ):
    return {
        'Authorization': 'Bearer ' + api_key,
        'Content-Type': 'application/json;charset=utf-8'
    }

    # 发送http请求
# def send_request(self, method, request_url, data):
proxies = {"http": "http://23.210.15.233:8080", "https": "https://23.210.15.233:8080"}
headers = get_req_header()
# if method == 'GET':
    # res = requests.get(request_url, params=data, headers=headers, verify=False)
# if method == 'POST':
#     res = requests.post(request_url, data=json.dumps(data), headers=headers, verify=False)
# result = res.json()

In [9]:
def send_request( method, request_url, data):
        proxies = {"http": "http://23.210.15.233:8080", "https": "https://23.210.15.233:8080"}
        headers =  get_req_header()
        if method == 'GET':
            res = requests.get(request_url, params=data, headers=headers, verify=False)
        if method == 'POST':
            res = requests.post(request_url, data=json.dumps(data), headers=headers, verify=False)
        result = res.json()
        if res.status_code >= 400:
            if result['code'] == 8303 :
                # 5s后重试
                time.sleep(5)
                return  send_request(method, request_url, data)
            else:
                raise Exception('请求错误！', result)
        else:
            return result

In [10]:
def get_form_data(dataId, limit, fields, data_filter):
        result = send_request('POST', url_get_data, {
            'data_id': dataId,
            'limit': limit,
            'fields': fields,
            'filter': data_filter
        })
        return result['data']

In [12]:
def get_all_data( fields, data_filter):
    form_data = []

    # 递归取下一页数据
    def get_next_page(dataId):
        data = get_form_data(dataId, 100, fields, data_filter)
        if data:
            for v in data:
                form_data.append(v)
            dataId = data[len(data) - 1]['_id']
            get_next_page(dataId)
    get_next_page('')
    return form_data

In [14]:

import pandas as pd
import jdy
import pgOperation


# In[3]:


def getFullJdyDf(APP_ID, ENTRY_ID, API_KEY):
    api = jdy.APIUtils(APP_ID, ENTRY_ID, API_KEY)
    form_data = api.get_all_data([], {})
    df3 = pd.json_normalize(form_data)
    return df3

In [15]:

def writePgFromApi(pg,APP_ID, ENTRY_ID, API_KEY, table_name):
    df = getFullJdyDf(APP_ID, ENTRY_ID, API_KEY)
    if table_name == 'huawei_daliy_operation':
        df.columns = ["deleter","create_time","update_time","delete_time","update_person","update_date","origin_receive","origin_send","origin_psn","honor_receive","honer_send","honor_psn","pearlriver_transport_times","guiyang_transport_times","t_product_receive","t_product_clean","t_product_send","r4_receive","r4_send","r4_psn","nanhua_receive","nanhua_send","nanhau_psn","anshi_receive","anshi_send","anshi_case","te_origin_receive","te_origin_send","te_product_receive","te_product_send","pre_receive","pre_withdraw","honor_transport_times","id","app_id","entry_id","creator_id","creator_name","creator_username","creator_status","updater_id","updater_name","updater_username","updater_status","creator","updater"]
    elif table_name == 'huawei_output':
        df.columns = ["creator","updater","deleter","create_time", "update_time", "delete_time", "update_date","origin_receive","origin_send","origin_psn","honor_receive","honer_send","honor_psn","pearlriver_transport_times","guiyang_transport_times","t_product_receive","t_product_clean","t_product_send","r4_receive","r4_send","r4_psn","nanhua_receive","nanhua_send","nanhau_psn","anshi_receive","anshi_send","anshi_case","te_origin_receive","te_origin_send","te_product_receive","te_product_send","pre_receive","pre_withdraw","honor_transport_times","id","app_id","entry_id"]
    elif table_name == 'huawei_outsourcing_name_list':
        df.columns = ["deleter","create_time","update_time","delete_time", 'area',
        'out_source_company', 'company', 'emp_no', 'emp_name', 'city',
        'job_name', 'site_name', 'cost_center', 'entry_date', 'leave_date',
        'is_rehired', 'is_cancel_entry', 'salary_amount', 'job_allowance',
        'meal_allowance', 'traffic_allowance', 'accom_allowance',
        'phone_allowance', 'social_insurance_allowance', 'level_allowance',
        'skill_allowance', 'env_allowance', 'special_allowance',
        'seniority_allowance', 'guaranteed_bonus', 'social_insurance_cost',
        'out_source_service_cost', 'package_salary', 'charged_hr',
        'update_time', 'comment', 'id_no', 'dept', 'by_time_or_piece',
        'emp_type', "id","app_id","entry_id","creator_id","creator_name",
        "creator_username","creator_status","updater_id","updater_name",
        "updater_username","updater_status"]
    elif table_name == 'huawei_opt_dept_mapping':
        df.columns = ["deleter","create_time","update_time","delete_time", 'flow_state',
        'table_id', 'mapping_no', 'site', 'dept', 'eff_index_name', 'ou_code',
        "id","app_id","entry_id","creator_id","creator_name",
        "creator_username","creator_status","updater_id","updater_name",
        "updater_username","updater_status"]

    # pg.writeDfToPg(df,table_name)
    # pg.grantAllOn(table_name,'smart')
    print(df.head())
    return df

In [16]:
APP_ID_1 = '617643f8308c2f000941f21b'
ENTRY_ID_1 = '617915c10613da0007c9a4d4'
API_KEY_1 = 'nqLEWktb51m3oYbYOZa5a2Ai6Lp8BHT7'
pg = pgOperation.PgOperation('public') 
writePgFromApi(pg,APP_ID_1, ENTRY_ID_1, API_KEY_1,'huawei_daliy_operation')

ConnectionError: HTTPSConnectionPool(host='www.jiandaoyun.com', port=443): Max retries exceeded with url: /api/v1/app/617643f8308c2f000941f21b/entry/617915c10613da0007c9a4d4/data (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000236959B6880>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))